### Quantum Fisher Information matrix

<img src="../images/qfim.png">

I have coded QFIM function and test it on 1qubit case

$\partial_\theta|\psi\rangle$ can be calculated by on the below equation (from Le Bin Ho):

<img src="../images/grad_psi.png" width=500px/>

The QNG are not simple as our imagin, the det of QFIM is 0 at almost step, so when using Moore-Penrose pseudo inverse the results're not good. The new solution is based on [this paper](https://arxiv.org/pdf/1909.02108.pdf). It seems to more complex, there are many that need to code:

- Create function that return the generator for every circuit

- Calculating the Fubini-Study tensor

- Divide circuit into some layer, and use recursion on it.

$R_X(\theta)=e^{i\frac{\theta}{2}X}$

In [1]:
import qiskit
import numpy as np
import qtm.base_qtm, qtm.constant, qtm.qtm_1qubit, qtm.qtm_nqubit
import matplotlib.pyplot as plt
from types import FunctionType

In [26]:
def create_circuit(qc, thetas):
    # |psi_0>: state preparation
    qc.ry(np.pi / 4, 0)
    qc.ry(np.pi / 4, 1)
    qc.ry(np.pi / 6, 2)
    # V0(theta0, theta1): Parametrized layer 0
    qc.rz(thetas[0], 0)
    qc.rz(thetas[1], 1)
    # W1: non-parametrized gates
    qc.cnot(0, 1)
    qc.cnot(1, 2)
    # V_1(theta2, theta3): Parametrized layer 1
    qc.ry(thetas[2], 1)
    qc.rx(thetas[3], 2)
    # W2: non-parametrized gates
    qc.cnot(0, 1)
    qc.cnot(1, 2)
    return qc

def create_psi():
    qc = qiskit.QuantumCircuit(3, 3)
    qc.ry(np.pi / 4, 0)
    qc.ry(np.pi / 3, 1)
    qc.ry(np.pi / 7, 2)
    return qc

generator_ry = np.array([
    [0, -1j],
    [1j, 0]
], dtype=np.complex128)


In [27]:

# calculate g_ij
qc = create_psi()
# psi_qc = qiskit.quantum_info.Statevector.from_instruction(qc).data
qc.measure(0, 0)
qc.measure(1, 1)
counts = qiskit.execute(qc, backend = qtm.constant.backend, shots = qtm.constant.num_shots).result().get_counts()

for i in counts:
    counts[i] /= qtm.constant.num_shots

print(counts)
p_q0_0 = sum([v for k, v in counts.items() if k[-1] == '0'])
p_q0_1 = sum([v for k, v in counts.items() if k[-1] == '1'])
p_q1_0 = sum([v for k, v in counts.items() if k[-2] == '0'])
p_q1_1 = sum([v for k, v in counts.items() if k[-2] == '1'])
p_q0q1_00 = sum([v for k, v in counts.items() if k[-2:] == '00'])
p_q0q1_01 = sum([v for k, v in counts.items() if k[-2:] == '10'])
p_q0q1_10 = sum([v for k, v in counts.items() if k[-2:] == '01'])
p_q0q1_11 = sum([v for k, v in counts.items() if k[-2:] == '11'])


z0 = p_q0_0 - p_q0_1
z0_2 = 1
z1 = p_q1_0 - p_q1_1
z1_2 = 1

z0_z1 = (p_q0_0 - p_q0_1)*(p_q1_0 - p_q1_1)
z0z1 = p_q0q1_00 - p_q0q1_01 - p_q0q1_10 + p_q0q1_11

g0 = np.zeros([2, 2])
g0[0, 0] = z0_2 - z0**2
g0[0, 1] = z0z1 - z0_z1
g0[1, 0] = z0z1 - z0_z1
g0[1, 1] = z1_2 - z1**2

qc.draw('mpl')

print(g0 / 4)


{'000': 0.6404, '010': 0.211, '001': 0.1108, '011': 0.0378}
[[0.12651804 0.00082832]
 [0.00082832 0.18689856]]


$|\psi_0\rangle=R_Y(\pi/4)\otimes R_Y(\pi/3)\otimes R_Y(\pi/7)$

So $|\psi\rangle$ in $p(ij)$ formula means $R_Y(\pi/4)\otimes R_Y(\pi/3)$?

In [24]:

# calculate g_ij
qc = create_psi()
# psi_qc = qiskit.quantum_info.Statevector.from_instruction(qc).data
qc.measure(0, 0)
counts = qiskit.execute(qc, backend = qtm.constant.backend, shots = qtm.constant.num_shots).result().get_counts()

for i in counts:
    counts[i] /= qtm.constant.num_shots

p_q0_0 = sum([v for k, v in counts.items() if k[-1] == '0'])
p_q0_1 = sum([v for k, v in counts.items() if k[-1] == '1'])

print('p_q0_0', p_q0_0)
print('p_q0_1', p_q0_1)

qc = create_psi()
qc.measure(1, 1)
counts = qiskit.execute(qc, backend = qtm.constant.backend, shots = qtm.constant.num_shots).result().get_counts()
for i in counts:
    counts[i] /= qtm.constant.num_shots

p_q1_0 = sum([v for k, v in counts.items() if k[-2] == '0'])
p_q1_1 = sum([v for k, v in counts.items() if k[-2] == '1'])

qc = create_psi()
qc.measure(0, 0)
qc.measure(1, 1)
counts = qiskit.execute(qc, backend = qtm.constant.backend, shots = qtm.constant.num_shots).result().get_counts()
for i in counts:
    counts[i] /= qtm.constant.num_shots

p_q0q1_00 = sum([v for k, v in counts.items() if k[-2:] == '00'])
p_q0q1_01 = sum([v for k, v in counts.items() if k[-2:] == '10'])
p_q0q1_10 = sum([v for k, v in counts.items() if k[-2:] == '01'])
p_q0q1_11 = sum([v for k, v in counts.items() if k[-2:] == '11'])


z0 = p_q0_0 - p_q0_1
z0_2 = 1
z1 = p_q1_0 - p_q1_1
z1_2 = 1

z0_z1 = (p_q0_0 - p_q0_1)*(p_q1_0 - p_q1_1)
z0z1 = p_q0q1_00 - p_q0q1_01 - p_q0q1_10 + p_q0q1_11

g0 = np.zeros([2, 2])
g0[0, 0] = z0_2 - z0**2
g0[0, 1] = z0z1 - z0_z1
g0[1, 0] = z0z1 - z0_z1
g0[1, 1] = z1_2 - z1**2

qc.draw('mpl')

print(g0 / 4)


p_q0_0 0.8497
p_q0_1 0.1503
[[0.12770991 0.00466765]
 [0.00466765 0.02389975]]


In [20]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=3)
g0 = np.zeros([2, 2])
params = np.array([0.432, -0.123, 0.543, 0.233])

def layer0_subcircuit(params):
    """This function contains all gates that
    precede parametrized layer 0"""
    qml.RY(np.pi / 4, wires=0)
    qml.RY(np.pi / 10, wires=1)
    qml.RY(np.pi / 6, wires=2)
@qml.qnode(dev)
def layer0_diag(params):
    layer0_subcircuit(params)
    return qml.var(qml.PauliZ(0)), qml.var(qml.PauliZ(1))


# calculate the diagonal terms
varK0, varK1 = layer0_diag(params)
g0[0, 0] = varK0 / 4
g0[1, 1] = varK1 / 4

print(g0)

[[0.125      0.        ]
 [0.         0.02387288]]
